# Оптимизация кода дифференциальных уравнений

В этом блокноте мы рассмотрим некоторые из основных инструментов для оптимизации вашего кода для эффективного решения DifrentialEquations.jl. Оптимизация на стороне пользователя важна, потому что для достаточно сложных задач большая часть времени будет проводиться внутри вашей функции `f`, функции, которую вы пытаетесь решить. «Эффективными» интеграторами являются те, которые уменьшают необходимое количество вызовов `f` для достижения допустимой ошибки. Основными идеями оптимизации кода DiffEq или любой функции Julia являются следующие: 

- сделать его нераспределенным 
- использовать StaticArrays для небольших массивов 
- использовать широковещательное слияние 
- сделать его устойчивым к типу 
- сократить избыточные вычисления 
- Используйте вызовы BLAS 
- Оптимизируйте выбор алгоритма 

Мы обсудим эти стратегии в контексте малых и больших систем. Начнем с небольших систем.

## Оптимизация малых систем (<100 ДУ)

Давайте возьмем ранеерасмотренную классическую систему Лоренца. Давайте начнем с наивного написания системы в неуместной форме:

In [ ]:
function lorenz(u,p,t)
 dx = 10.0*(u[2]-u[1])
 dy = u[1]*(28.0-u[3]) - u[2]
 dz = u[1]*u[2] - (8/3)*u[3]
 [dx,dy,dz]
end

Здесь `lorenz` возвращает объект` [dx, dy, dz] `, который создан внутри тела` lorenz`. 

Это обычный шаблон кода из языков высокого уровня, таких как MATLAB, SciPy или R's deSolve. Однако проблема этой формы заключается в том, что она выделяет вектор `[dx, dy, dz]` на каждом шаге. Давайте оценим процесс решения с помощью этого выбора функции:

In [ ]:
using DifferentialEquations, BenchmarkTools, LinearAlgebra

In [ ]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
@benchmark solve(prob,Tsit5())

Макрос `Benchmark` пакета BenchmarkTools запускает код несколько раз, чтобы получить точное измерение. Минимальное время - это время, которое требуется, когда ваша ОС и другие фоновые процессы не мешают. Обратите внимание, что в этом случае требуется около 6 мс для решения и выделяет около 11,11 МБ. Однако, если бы мы использовали это внутри реального пользовательского кода, мы бы увидели много времени, потраченного на сборку мусора (GC) для очистки всех созданных нами массивов. Даже если мы отключим сохранение, у нас есть эти распределения.

In [ ]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

Проблема, конечно, в том, что массивы создаются каждый раз, когда вызывается наша производная функция. Эта функция вызывается несколько раз за шаг и, таким образом, является основным источником использования памяти. Чтобы исправить это, мы можем использовать форму на месте (in-place - изменение поступающего объекта без создания нового), чтобы ***сделать наш код невыделенным***:

In [ ]:
function lorenz!(du,u,p,t)
 du[1] = 10.0*(u[2]-u[1])
 du[2] = u[1]*(28.0-u[3]) - u[2]
 du[3] = u[1]*u[2] - (8/3)*u[3]
end

Здесь вместо создания массива каждый раз мы использовали кеш-массив `du`. Когда используется изменение поступающих данных (форма на месте), DifferentialEquations.jl использует другой внутренний маршрут, который также минимизирует внутренние распределения. Когда мы оценим эту функцию, мы увидим разницу.

In [ ]:
u0 = [1.0;0.0;0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan)
@benchmark solve(prob,Tsit5())

In [ ]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

Разница в 4 раза! Обратите внимание, что есть еще некоторые выделения памяти, и это связано с созданием кеша интеграции. Но это не масштабируется с размером проблемы:

In [ ]:
tspan = (0.0,500.0) # 5x longer than before
prob = ODEProblem(lorenz!,u0,tspan)
@benchmark solve(prob,Tsit5(),save_everystep=false)

так как это все неизбежные затраты.

####  Но если система маленькая, мы можем оптимизировать еще больше.

Выделения памяти являются дорогостоящими, только если они являются «распределением кучи». Для более глубокого определения распределения кучи [есть много источников в сети](http://net-informations.com/faq/net/stack-heap.htm). Но хорошим рабочим определением является то, что выделение кучи - это куски памяти переменного размера, на которые нужно указывать, и это косвенное обращение к указателю стоит времени. Кроме того, куча должна управляться и контроллеры мусора должны активно отслеживать, что находится в куче. 

Однако есть альтернатива распределению в куче, известная как распределение в стеке. Стек имеет статический размер (известный во время компиляции) и, следовательно, доступ к нему быстрый. Кроме того, точный блок памяти заранее известен компилятору, поэтому повторное использование памяти обходится дешево. Это означает, что размещение в стеке практически не требует затрат! 

Массивы должны быть выделены в куче, потому что их размер (и, следовательно, объем занимаемой ими памяти) определяется во время выполнения. Но у Юлии есть структуры, которые размещаются в стеке. `struct`, например, это "тип значения", выделенный стеком. `Tuple`s - это выделенная стеком коллекция. Наиболее полезная структура данных для DiffEq - это `StaticArray` из пакета [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl). Длина этих массивов определяется во время компиляции. Они создаются с помощью макросов, прикрепленных к выражениям обычного массива, например:

In [ ]:
using StaticArrays
A = @SVector [2.0,3.0,5.0]

Обратите внимание, что `3` после` SVector` дает размер `SVector`. Он не может быть изменен. Кроме того, `SVector`а являются неизменяемыми, поэтому мы должны создать новый` SVector` для изменения значений. Но помните, нам не нужно беспокоиться о распределении, потому что эта структура данных размещена в стеке. У SArray также есть много дополнительных оптимизаций: у них быстрое умножение матриц, быстрая QR-факторизация и т.д., которые напрямую используют информацию о размере массива. Таким образом, когда это возможно, их следует использовать. 

К сожалению, статические массивы можно использовать только для достаточно маленьких массивов. После определенного размера они вынуждены распределять кучу после некоторых инструкций и их всплывающих подсказок. Таким образом, статические массивы не должны использоваться, если в вашей системе более 100 переменных. Кроме того, только нативные алгоритмы Julia могут полностью использовать статические массивы. 

Давайте ***оптимизируем `lorenz`, используя статические массивы***. Обратите внимание, что в этом случае мы хотим использовать форму размещения вне места, но на этот раз мы хотим вывести статический массив:

In [ ]:
function lorenz_static(u,p,t)
 dx = 10.0*(u[2]-u[1])
 dy = u[1]*(28.0-u[3]) - u[2]
 dz = u[1]*u[2] - (8/3)*u[3]
 @SVector [dx,dy,dz]
end

Чтобы заставить решатель использовать статические массивы внутри, мы просто даем ему статический массив в качестве начального условия:

In [ ]:
u0 = @SVector [1.0,0.0,0.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz_static,u0,tspan)
@benchmark solve(prob,Tsit5())

In [ ]:
@benchmark solve(prob,Tsit5(),save_everystep=false)

И это почти все, что нужно сделать. Со статическими массивами вам не нужно беспокоиться о распределении, поэтому используйте такие операции, как `*`, и не беспокойтесь о операциях слияния (обсуждается в следующем разделе). Сделайте «векторизованный код» как в R / MATLAB / Python(NumPY) и ваш код в этом случае будет быстрым, или напрямую используйте числа / значения.

####  Упражнение 1 

Проделайте все вышеперечисленные выкрутасы для [Системы Хенона-Хейлса](https://en.wikipedia.org/wiki/H%C3%A9non%E2%80%93Heiles_system) и проведите бенчмаркинг.

## Оптимизация больших систем

### Интерлюдия: Управление распределением с помощью Broadcast Fusion

Если ваша система достаточно велика или вам приходится использовать не родной алгоритм Джулии, вы должны использовать `Array`сы. Чтобы использовать массивы наиболее эффективно, вам нужно быть осторожным с временными выделениями памяти. Векторизованные вычисления, естественно, имеют много временных массивов. Это потому, что векторизованный расчет выводит вектор. Таким образом:

In [ ]:
A = rand(1000,1000); B = rand(1000,1000); C = rand(1000,1000)
test(A,B,C) = A + B + C
@benchmark test(A,B,C)

Это выражение `A + B + C` создает 2 массива. Сначала он создает его для вывода `A + B`, затем использует этот массив результатов для` + C`, чтобы получить окончательный результат. 2 массива! Мы этого не хотим! Первое, что нужно сделать, чтобы это исправить - использовать широковещательный синтез. [Broadcast Fusion](https://julialang.org/blog/2017/01/moredots) объединяет выражения. Например, вместо выполнения операций `+` по отдельности, если бы мы добавили их все одновременно, то у нас был бы только один созданный массив. Например:

In [ ]:
test2(A,B,C) = map((a,b,c)->a+b+c,A,B,C)
@benchmark test2(A,B,C)

Размещение всего выражения в один вызов функции требует для хранения выходных данных только один массив. Это то же самое, что написать цикл:

In [ ]:
function test3(A,B,C)
    D = similar(A)
    @inbounds for i in eachindex(A)
        D[i] = A[i] + B[i] + C[i]
    end
    D
end
@benchmark test3(A,B,C)

Однако бродкаст Юлии имеет свой синтаксический сахар. Если у выражений приписана `.`, тогда они объединят эти векторизованные операции. Таким образом:

In [ ]:
test4(A,B,C) = A .+ B .+ C
@benchmark test4(A,B,C)

версия с созданным только 1 массивом (вывод) Обратите внимание, что `.` также может использоваться с вызовами функций:

In [ ]:
sin.(A) .+ sin.(B)

Также макрос `@.` применяет точку к каждому оператору:

In [ ]:
test5(A,B,C) = @. A + B + C #only one array allocated
@benchmark test5(A,B,C)

Используя эти инструменты, мы можем избавиться от распределения промежуточных массивов для многих вызовов векторизованных функций. Но мы все еще выделяем выходной массив. Чтобы избавиться от этого распределения, мы можем вместо этого использовать мутацию. Мутирующая трансляция осуществляется через `.=`. Например, если мы предварительно определили вывод:

In [ ]:
D = zeros(1000,1000);

Затем мы можем продолжать использовать этот кэш для последующих вычислений. Форма мутирующего широковещания:

In [ ]:
test6!(D,A,B,C) = D .= A .+ B .+ C #only one array allocated
@benchmark test6!(D,A,B,C)

Если мы будем использовать `@.` перед` = `, тогда он превратится в `.= `:

In [ ]:
test7!(D,A,B,C) = @. D = A + B + C #only one array allocated
@benchmark test7!(D,A,B,C)

Обратите внимание, что в этом случае нет «выходных данных», и вместо этого значения внутри `D` - это то, что изменяется (как в случае с функцией DiffEq in-place). Многие функции Юлии имеют мутирующую форму, которая обозначается символом `!`. Например, мутирующая форма `map` это` map!`:

In [ ]:
test8!(D,A,B,C) = map!((a,b,c)->a+b+c,D,A,B,C)
@benchmark test8!(D,A,B,C)

Некоторые операции требуют использования альтернативной мутирующей формы для быстрой работы. Например, умножение матриц через `*` породит временное выделение памяти:

In [ ]:
@benchmark A*B

Вместо этого мы можем использовать мутирующую форму `A_mul_B!` В массиве кеша, чтобы избежать выделения памяти:

In [ ]:
@benchmark A_mul_B!(D,A,B) # same as D = A * B сами определяйте

Для повторных вычислений это сокращенное распределение может остановить циклы GC и таким образом привести к более эффективному коду. Кроме того, ***мы можем объединить операции линейной алгебры более высокого уровня, используя BLAS***. Пакет [SugarBLAS.jl](https://github.com/lopezm94/SugarBLAS.jl) позволяет легко писать операции более высокого уровня, такие как `alpha * B * A + beta * C`, в качестве мутирующих вызовов BLAS.

### Пример оптимизации: дискретизация реакции Гирера-Майнхардта - диффузионного УвЧП

Оптимизируем решение реакционно-диффузионного PDE. В своей дискретной форме это ODE:

$$ du = D_1 (A_y u + u A_x) + \frac{au^2}{v} + \bar{u} - \alpha u $$
$$ dv = D_2 (A_y v + v A_x) + a u^2 + \beta v $$

где $ u $, $ v $ и $ A $ - матрицы. Здесь мы будем использовать упрощенную версию, где $ A $ - это трехдиагональный трафарет $ [1, -2,1] $, то есть это двумерная дискретизация Лапласиана. Нативный код будет чем-то вроде:

In [ ]:
# Generate the constants
p = (1.0,1.0,1.0,10.0,0.001,100.0) # a,α,ubar,β,D1,D2
N = 100
Ax = Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1])
Ay = copy(Ax)
Ax[2,1] = 2.0
Ax[end-1,end] = 2.0
Ay[1,2] = 2.0
Ay[end,end-1] = 2.0                                

function basic_version!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = r[:,:,1]
  v = r[:,:,2]
  Du = D1*(Ay*u + u*Ax)
  Dv = D2*(Ay*v + v*Ax)
  dr[:,:,1] = Du .+ a*u*u./v .+ ubar - α*u
  dr[:,:,2] = Dv .+ a*u*u - β*v   
end

a,α,ubar,β,D1,D2 = p
uss = (ubar+β)/α
vss = (a/β)*uss^2
r0 = zeros(100,100,2)
r0[:,:,1] .= uss+0.1*rand()
r0[:,:,2] .= vss

prob = ODEProblem(basic_version!,r0,(0.0,0.1),p)

В этой версии мы закодировали наше начальное условие как трехмерный массив, где `u [:,:, 1]` является частью `A`, а` u [:,:, 2] `является частью ` B`.

In [ ]:
@benchmark solve(prob,Tsit5())

хотя эта версия не очень эффективна,

#### Мы рекомендуем сначала написать «высокоуровневый» код и итеративно оптимизировать его! 

Первое, что мы можем сделать, это избавиться от выделения срезов. Операция `r[:,:, 1]` создает временный массив вместо «представления», то есть указатель на уже существующую память. Чтобы создать представление (view), добавьте `
@view`. Обратите внимание, что мы должны быть осторожны с представлениями, потому что они указывают на одну и ту же память, и, следовательно, изменение представления изменяет исходные значения:

In [ ]:
A = rand(4)
@show A
B = @view A[1:3]
B[2] = 2
@show A

Обратите внимание, что изменение `B` изменило` A`. Так что нужно соблюдать осторожность. Мы хотим изменить вывод `dr`. Кроме того, последнее утверждение является чисто поэлементной операцией, и, таким образом, мы можем использовать там слияние трансляций. Давайте перепишем `basic_version!` стараясь ***избежать промежуточных выделений памяти*** и ***используя широковещательный синтез***:

In [ ]:
function gm2!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  Du = D1*(Ay*u + u*Ax)
  Dv = D2*(Ay*v + v*Ax)
  @. du = Du + a*u*u./v + ubar - α*u
  @. dv = Dv + a*u*u - β*v   
end
prob = ODEProblem(gm2!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

Теперь большая часть выделений памяти происходит в `Du = D1 * (Ay * u + u * Ax)`, поскольку эти операции векторизованы, а не мутируют. Вместо этого мы должны заменить матричные умножения на `mul!`. При этом нам понадобятся переменные кэша для записи. Это выглядит так:

In [ ]:
Ayu = zeros(N,N)
uAx = zeros(N,N)
Du = zeros(N,N)
Ayv = zeros(N,N)
vAx = zeros(N,N)
Dv = zeros(N,N)
function gm3!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  mul!(Ayu,Ay,u)  # A_mul_B! в старой версии
  mul!(uAx,u,Ax)
  mul!(Ayv,Ay,v)
  mul!(vAx,v,Ax)
  @. Du = D1*(Ayu + uAx)
  @. Dv = D2*(Ayv + vAx)
  @. du = Du + a*u*u./v + ubar - α*u
  @. dv = Dv + a*u*u - β*v   
end
prob = ODEProblem(gm3!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

Но наши временные переменные являются глобальными переменными. Нам нужно либо объявить кэши как const, либо локализовать их. Мы можем локализовать их, добавив их к параметрам `p`. Компилятору проще рассуждать о локальных переменных, чем о глобальных. ***Локализация переменных помогает обеспечить стабильность типа***.

In [ ]:
p = (1.0,1.0,1.0,10.0,0.001,100.0,Ayu,uAx,Du,Ayv,vAx,Dv) # a,α,ubar,β,D1,D2
function gm4!(dr,r,p,t)
  a,α,ubar,β,D1,D2,Ayu,uAx,Du,Ayv,vAx,Dv = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  mul!(Ayu,Ay,u)
  mul!(uAx,u,Ax)
  mul!(Ayv,Ay,v)
  mul!(vAx,v,Ax)
  @. Du = D1*(Ayu + uAx)
  @. Dv = D2*(Ayv + vAx)
  @. du = Du + a*u*u./v + ubar - α*u
  @. dv = Dv + a*u*u - β*v   
end
prob = ODEProblem(gm4!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

Затем мы могли бы использовать BLAS `gemmv` для еще большей оптимизации умножения матриц, но вместо этого давайте удалим трафарет.

In [ ]:
p = (1.0,1.0,1.0,10.0,0.001,100.0,N)
function fast_gm!(du,u,p,t)
  a,α,ubar,β,D1,D2,N = p

  @inbounds for j in 2:N-1, i in 2:N-1
    du[i,j,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end

  @inbounds for j in 2:N-1, i in 2:N-1
    du[i,j,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
            a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds for j in 2:N-1
    i = 1
    du[1,j,1] = D1*(2u[i+1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
            a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for j in 2:N-1
    i = 1
    du[1,j,2] = D2*(2u[i+1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
            a*u[i,j,1]^2 - β*u[i,j,2]
  end
  @inbounds for j in 2:N-1
    i = N
    du[end,j,1] = D1*(2u[i-1,j,1] + u[i,j+1,1] + u[i,j-1,1] - 4u[i,j,1]) +
           a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for j in 2:N-1
    i = N
    du[end,j,2] = D2*(2u[i-1,j,2] + u[i,j+1,2] + u[i,j-1,2] - 4u[i,j,2]) +
           a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds for i in 2:N-1
    j = 1
    du[i,1,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for i in 2:N-1
    j = 1
    du[i,1,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
              a*u[i,j,1]^2 - β*u[i,j,2]
  end
  @inbounds for i in 2:N-1
    j = N
    du[i,end,1] = D1*(u[i-1,j,1] + u[i+1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
  end
  @inbounds for i in 2:N-1
    j = N
    du[i,end,2] = D2*(u[i-1,j,2] + u[i+1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]
  end

  @inbounds begin
    i = 1; j = 1
    du[1,1,1] = D1*(2u[i+1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
              a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[1,1,2] = D2*(2u[i+1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
              a*u[i,j,1]^2 - β*u[i,j,2]

    i = 1; j = N
    du[1,N,1] = D1*(2u[i+1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[1,N,2] = D2*(2u[i+1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]

    i = N; j = 1
    du[N,1,1] = D1*(2u[i-1,j,1] + 2u[i,j+1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[N,1,2] = D2*(2u[i-1,j,2] + 2u[i,j+1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]

    i = N; j = N
    du[end,end,1] = D1*(2u[i-1,j,1] + 2u[i,j-1,1] - 4u[i,j,1]) +
             a*u[i,j,1]^2/u[i,j,2] + ubar - α*u[i,j,1]
    du[end,end,2] = D2*(2u[i-1,j,2] + 2u[i,j-1,2] - 4u[i,j,2]) +
             a*u[i,j,1]^2 - β*u[i,j,2]
   end
end
prob = ODEProblem(fast_gm!,r0,(0.0,0.1),p)
@benchmark solve(prob,Tsit5())

Наконец, мы можем добавить еще плюшек, таких как многопоточность основных циклов, но это уже погоня за микросекундами. Основные оптимизации, которые применяются повсеместно, - это те, которые мы только что выполнили (хотя последняя работает только в том случае, если ваша матрица является трафаретом. Это называется реализацией дискретизации PDE без матрицы). 

Это дает нам примерно в 20 000 раз быстрее, чем наш оригинальный код векторизованного стиля MATLAB / SciPy / R! 

Последнее, что нужно сделать, - это ***оптимизировать наш алгоритм выбора***. Мы использовали `Tsit5()` в качестве нашего тестового алгоритма, но в действительности эта проблема представляет собой жесткую дискретизацию PDE, и поэтому одной из рекомендаций является использование `CVODE_BDF()`. Однако вместо использования плотного якобиана по умолчанию мы должны использовать разреженный якобиан, предоставленный проблемой. Якобиан - это матрица $ \frac{df_i}{dr_j} $, где $ r $ читается по линейному индексу (то есть по столбцам). Но так как переменные $ u $ зависят от $ v $, размерность велика, а якобиан полон нелинейностей и, следовательно, матрица якоби не очень поможет (~~я плохой переводчик~~). Вместо этого мы используем разреженные алгоритмы Якоби. `CVODE_BDF` позволяет нам использовать разреженный решатель Ньютона-Крылова путем установки` linear_solver =:GMRES` (см. [Документацию решателя](http://docs.juliadiffeq.org/latest/solvers/dae_solve.html#Sundials.jl-1), и, таким образом, мы можем эффективно решить эту проблему. Посмотрим, как это масштабируется при увеличении времени интеграции.

In [ ]:
prob = ODEProblem(fast_gm!,r0,(0.0,10.0),p)
@benchmark solve(prob,Tsit5())

In [ ]:
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES))

In [ ]:
prob = ODEProblem(fast_gm!,r0,(0.0,100.0),p)
@benchmark solve(prob,Tsit5())

In [ ]:
@benchmark solve(prob,CVODE_BDF(linear_solver=:GMRES))

Так как проблема жесткая, число шагов, требуемых явным методом Рунге-Кутты, быстро растет, тогда как `CVODE_BDF` предпринимает большие шаги. Кроме того, форма линейного решателя GMRES является довольно эффективным способом решения неявной системы в этом случае. Это зависит от проблемы, и во многих случаях для эффективного использования метода Крылова требуется предварительное исследование, поэтому вам нужно поэкспериментировать с тестированием других алгоритмов и линейных решателей, чтобы выяснить, что лучше всего подходит для вашей задачи. 

### Заключение 

Главное, что следует избегать - это временные выделения памяти. Для небольших систем это эффективно выполняется через статические массивы. Для больших систем это делается с помощью операций на месте и массивов кеша. В любом случае, полученное решение может быть значительно ускорено по векторизованным составам с использованием этих принципов.